使用R接口调用JAGS实现。

# 调用必要的包

In [3]:
install.packages("MCMCvis")

also installing the dependency 'overlapping'





The downloaded binary packages are in
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpPq8Nq7/downloaded_packages


In [17]:
install.packages("R2jags")

also installing the dependency 'R2WinBUGS'





The downloaded binary packages are in
	/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpPq8Nq7/downloaded_packages


In [18]:
library(rjags)
library(R2jags)
library(coda)
library(lattice)
library(MCMCvis)


Attaching package: 'R2jags'


The following object is masked from 'package:coda':

    traceplot




# 单样本无先验

In [20]:
jags.model.coin <- function() {
  Y ~ dbin(0.5, 10)
  P8 <- ifelse(Y>7, 1, 0)
}

In [30]:
# 模型拟合
jags.model.coin.fit <- jags(
  data = list(),
  model.file = jags.model.coin,
  parameters.to.save = c("Y", "P8"),
  n.iter = 10000,
  n.burnin = 1000,
  n.chains = 3,
  DIC = FALSE
)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 0
   Unobserved stochastic nodes: 1
   Total graph size: 8

Initializing model



In [31]:
print(jags.model.coin.fit)

Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpPq8Nq7/model17be914855946.txt", fit using jags,
 3 chains, each with 10000 iterations (first 1000 discarded), n.thin = 9
 n.sims = 3000 iterations saved. Running time = 0.012 secs
   mu.vect sd.vect 2.5% 25% 50% 75% 97.5%  Rhat n.eff
P8   0.052   0.223    0   0   0   0     1 1.008  1200
Y    4.987   1.580    2   4   5   6     8 1.003  1000

For each parameter, n.eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor (at convergence, Rhat=1).


# 单次数据有先验

In [32]:
## 模型
jags.mod.drug2 <- function(){
theta ~ dbeta(9.2,13.8) # 先验概率
r ~ dbin(theta,n) # 已有的实验数据
r.pred ~ dbin(theta,40) # 预测新的实验
P.crit <- ifelse(r.pred>=25,1,0) # 新实验大于25的概率
}
## 数据
r = 15 ## 实验有15个良性
n = 20 ## 实验进行了20次
jags.data.drug2 <- list('r','n') ## 把他俩添加为一个list
## 拟合
jags.mod.fit.drug2 <- jags(data = jags.data.drug2,
parameters.to.save =c('theta','r.pred','P.crit'),
n.chains = 2,n.iter = 10000,
model.file = jags.mod.drug2)
## data 输入变量r和n
## 一共五个变量，r和n是在data里输入的，而剩下三个是需要计算输出的。
## 输出结果
print(jags.mod.fit.drug2)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 1
   Unobserved stochastic nodes: 2
   Total graph size: 12

Initializing model

Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpPq8Nq7/model17be977dd2f70.txt", fit using jags,
 2 chains, each with 10000 iterations (first 5000 discarded), n.thin = 5
 n.sims = 2000 iterations saved. Running time = 0.096 secs
         mu.vect sd.vect   2.5%    25%    50%    75%  97.5%  Rhat n.eff
P.crit     0.336   0.472  0.000  0.000  0.000  1.000  1.000 1.001  2000
r.pred    22.672   4.327 14.000 20.000 23.000 26.000 31.000 1.001  2000
theta      0.564   0.075  0.414  0.514  0.564  0.614  0.709 1.001  2000
deviance   6.614   2.392  3.364  4.850  6.197  7.878 12.481 1.001  2000

For each parameter, n.eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor (at convergence, Rhat=1).

DIC info (using the rul

# 多次数据有先验

In [34]:
y <- c(rep(0,times = 229),rep(1,times=211),rep(2,times=93),
rep(3,times=35),rep(4,times=7),rep(7,times=1))
## 把一共 537个数据都写进y里去，被炸的次数这里用数字表示，而区域的数量这个里用数字出现的频数表示。
N <- length(y)
bombs.data <- list('y','N')

bombs.data

[[1]]
[1] "y"

[[2]]
[1] "N"

In [35]:
# 定义函数
jags.mod.bombs <- function(){
  theta ~ dgamma(0.5, 0.00001)
  for (i in 1:N) {
    y[i] ~ dpois(theta)
  }
}

In [36]:
## 设置了一个初始值，告诉JAGS从这里开始算，因为这是我根据泊松分布算出来的。
jags.booms.inits <- list('theta' = 537/576)
jags.inits.boom <- list(jags.booms.inits)

In [38]:
jags.mod.fit.boom <- jags(
  data = bombs.data,
  parameters.to.save = c('theta'),
  n.chains = 1,
  n.iter = 10000,
  model.file = jags.mod.bombs,
  inits = jags.inits.boom,
  DIC = FALSE
)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 576
   Unobserved stochastic nodes: 1
   Total graph size: 580

Initializing model



In [39]:
print(jags.mod.fit.boom)

Inference for Bugs model at "/var/folders/fv/c0dh9y6s7638rh5fdr3lgcmr0000gn/T//RtmpPq8Nq7/model17be97869663c.txt", fit using jags,
 1 chains, each with 10000 iterations (first 5000 discarded), n.thin = 5
 n.sims = 1000 iterations saved. Running time = 0.046 secs
      mu.vect sd.vect  2.5%   25%   50%  75% 97.5%
theta   0.933   0.041 0.851 0.906 0.932 0.96 1.012
